# Here you test something

Cols meaning:
- Day - day of the month
- DKind - day kind
- EC - exercise count
- EI - exercise intensity - has six levels from 0 - didn't exercise to 5 - hard exercise (More info in a separate message)
- OD - outdoor
- LE - left-eye reading
- RN - running
- GM - gaming


(Separate message about EI)
Exercise intensity notation:
- 0 - you didn't exercise
- 1 - easy
- 2 - between easy and moderate
- 3 - moderate
- 4 - between moderate and hard
- 5 - hard

In [1]:
import numpy as np
import pandas as pd

from preparation.prepare_data import prepare_data
from constants import PATH, APPEND_PATH, PREVIOUS_DATA_FOLDER

DAY_OFFS = []
DESIRED_MEAN_VALUE = 275
rep_data, wide_use_data, feedback_data = prepare_data(PATH, DAY_OFFS,
                                              DESIRED_MEAN_VALUE)
month, mean, std, df = wide_use_data
df

,Day,DKind,Math,CS,Eng,Total,EC,EI,OD,LE,RN,GM
0,1,work,0,0,43,43,1,2,130,0,0,0
1,2,work,0,265,15,280,1,0,105,0,0,0
2,3,work,0,205,69,274,2,3,165,10,0,0
3,4,work,0,185,115,300,3,5,230,0,13,0
4,5,work,0,0,19,19,4,3,160,10,0,0
5,6,work,0,260,66,326,5,4,210,0,11,0
6,7,work,0,195,114,309,6,4,140,10,0,0
7,8,work,0,50,10,60,7,2,215,0,0,0
8,9,work,0,250,30,280,7,0,120,10,0,0
9,10,work,0,222,22,244,8,4,290,0,15,0


In [2]:
rep_data

([0, 54, 17, 71, 203, 0, 'you studied every day'],
 [['OD', 'LE', 'RN', 'GM'],
  16,
  [65.83, 1.67, 1.25, 12.83],
  [20, 10, 6, 2],
  {1: 0, 2: 2, 3: 5, 4: 6, 5: 3}])

# Working with 'hobby' data

Get the file content

In [3]:
import re
import pandas as pd

PATH = "test.txt"
# PATH = "2023 - What I read, watched and played.txt"
# Read the text file with UTF-8 encoding
with open(PATH, "r", encoding="utf-8") as file:
    content = file.read()

print(content)

Books:
1. "The Republic of Thieves: Gentleman Bastards" by Scott Lynch | audiobook on January 5, 2023
2. "Hyperion" by Dan Simmons | audiobook on January 13, 2023
3. "Propaganda" by Edward Bernays | audiobook on January 17, 2023
4. "The Fall of Hyperion" by Dan Simmons | audiobook on January 23, 2023
5. "Джури-характерники" by Володимир Рутківський on January 23, 2023
6. "Zen in the Art of Writing" by Ray Bradbury | audiobook on January 25, 2023
7. "The Final Empire: Mistborn, Book 1" by Brandon Sanderson | audiobook on February 2, 2023
8. "The Well of Ascension: Mistborn, Book 2" by Brandon Sanderson | audiobook on February 4, 2023
9. "The Hero of Ages: Mistborn: Book 3" by Brandon Sanderson | audiobook on February 6, 2023
10. "Factotum" by Charles Bukowski | audiobook on February 9, 2023
11. "Ham on Rye" by Charles Bukowski | audiobook on February 12, 2023
12. "Warbreaker" by Brandon Sanderson on February 13, 2023
13. "Hollywood" by Charles Bukowski | audiobook on February 16, 2023
1

Get movie df

In [4]:
# Find the 'Movies:' section using the regular expression
movies_section = re.search(r"Movies:(.*?)(?=\n\n|$)", content, re.DOTALL).group(1).strip()

# Print the extracted movies section
print("Movies Section:")
print(movies_section)

Movies Section:
1. Romancing the Stone on Jan 6, 2023
2. Star Trek: The Wrath of Khan on Jan 15, 2023
3. Greyhound on Jan 16, 2023
4. The Thing on Jan 19, 2023


In [5]:
# Define a regular expression pattern for extracting movie information
movie_pattern = re.compile(r'\d+\. ([^\n]+) on ([A-Za-z]+) (\d{1,2}), (\d{4})')

# Extract movie information using the pattern
movie_data = movie_pattern.findall(movies_section)

cols = ["Name", "Month", "Day", "Year"]
# Create a pandas DataFrame for movies
movies_df = pd.DataFrame(movie_data, columns=cols)

# Print the DataFrame
print("Movies DataFrame:")
movies_df

Movies DataFrame:


,Name,Month,Day,Year
0,Romancing the Stone,Jan,6,2023
1,Star Trek: The Wrath of Khan,Jan,15,2023
2,Greyhound,Jan,16,2023
3,The Thing,Jan,19,2023


Get games df

In [6]:
# Find the 'Games:' section using the regular expression
games_section = re.search(r"Games:(.*?)$", content, re.DOTALL).group(1).strip()

# Define a regular expression pattern for extracting game information
game_pattern = re.compile(r'\d+\. ([^\n]+) on ([A-Za-z]+) (\d{1,2}), (\d{4}) \((\d+) hours\)')

# Extract game information using the pattern
game_data = game_pattern.findall(games_section)

# 'PTH' stands for 'Play Time in Hours'
cols=["Name", "Month", "Day", "Year", "PTH"]
# Create a pandas DataFrame for games
games_df = pd.DataFrame(game_data, columns=cols)

# Print the DataFrame
print("Games DataFrame:")
games_df

Games DataFrame:


,Name,Month,Day,Year,PTH
0,Marvel's Guardians of the Galaxy,Jan,15,2023,25
1,Titanfall 2,February,19,2023,6


Get books df

In [7]:
# Find the 'Books:' section using string manipulation
books_section = re.search(r"Books:(.*?)(?=Movies:|Games:)", content, re.DOTALL).group(1).strip()
# Split the section into lines
book_lines = books_section.split('\n')
book_lines

['1. "The Republic of Thieves: Gentleman Bastards" by Scott Lynch | audiobook on January 5, 2023',
 '2. "Hyperion" by Dan Simmons | audiobook on January 13, 2023',
 '3. "Propaganda" by Edward Bernays | audiobook on January 17, 2023',
 '4. "The Fall of Hyperion" by Dan Simmons | audiobook on January 23, 2023',
 '5. "Джури-характерники" by Володимир Рутківський on January 23, 2023',
 '6. "Zen in the Art of Writing" by Ray Bradbury | audiobook on January 25, 2023',
 '7. "The Final Empire: Mistborn, Book 1" by Brandon Sanderson | audiobook on February 2, 2023',
 '8. "The Well of Ascension: Mistborn, Book 2" by Brandon Sanderson | audiobook on February 4, 2023',
 '9. "The Hero of Ages: Mistborn: Book 3" by Brandon Sanderson | audiobook on February 6, 2023',
 '10. "Factotum" by Charles Bukowski | audiobook on February 9, 2023',
 '11. "Ham on Rye" by Charles Bukowski | audiobook on February 12, 2023',
 '12. "Warbreaker" by Brandon Sanderson on February 13, 2023',
 '13. "Hollywood" by Charles 

In [8]:
# Remove all data before the first double quote in each line
book_lines = [line.split('"', 1)[1] for line in book_lines]
book_lines

['The Republic of Thieves: Gentleman Bastards" by Scott Lynch | audiobook on January 5, 2023',
 'Hyperion" by Dan Simmons | audiobook on January 13, 2023',
 'Propaganda" by Edward Bernays | audiobook on January 17, 2023',
 'The Fall of Hyperion" by Dan Simmons | audiobook on January 23, 2023',
 'Джури-характерники" by Володимир Рутківський on January 23, 2023',
 'Zen in the Art of Writing" by Ray Bradbury | audiobook on January 25, 2023',
 'The Final Empire: Mistborn, Book 1" by Brandon Sanderson | audiobook on February 2, 2023',
 'The Well of Ascension: Mistborn, Book 2" by Brandon Sanderson | audiobook on February 4, 2023',
 'The Hero of Ages: Mistborn: Book 3" by Brandon Sanderson | audiobook on February 6, 2023',
 'Factotum" by Charles Bukowski | audiobook on February 9, 2023',
 'Ham on Rye" by Charles Bukowski | audiobook on February 12, 2023',
 'Warbreaker" by Brandon Sanderson on February 13, 2023',
 'Hollywood" by Charles Bukowski | audiobook on February 16, 2023',
 'Pulp" by C

In [15]:
import re

date_string = 'on January 15th, 2023'

# Use regex to extract month, day, and year
match = re.match(r'on ([A-Za-z]+) (\d{1,2})[a-z]{2}, (\d{4})', date_string)
print(match)
if match:
    month = match.group(1)[:3]
    day = int(match.group(2))
    year = int(match.group(3))
    result = {'Month': month, 'Day': day, 'Year': year}
    print(result)

<re.Match object; span=(0, 21), match='on January 15th, 2023'>
{'Month': 'Jan', 'Day': 15, 'Year': 2023}


In [16]:
# Initialize variables to store book data
book_data = []
cur_book = {}
audiobook_marker = "| audiobook"
# Process each line in the book section
for line in book_lines:
    print(line)
    if audiobook_marker in line:
        # Audiobook entry
        # Split the data on book data (book name and author)
        # and date (Month, day, and year)
        name_author, date = line.split(audiobook_marker)
        print(name_author, date_info)
        name, author = name_author.split('" by ')
        print(name, author)
        # Use 'strip()' to remove trailing whitespaces
        cur_book["Name"] = name.strip()
        cur_book["Author"] = author.strip()
        cur_book["Audiobook"] = True
        print(cur_book)
        # Extract date info using regex
        match = re.match(r'on ([A-Za-z]+) (\d{1,2})[a-z]{2}, (\d{4})', date_string)
        cur_book["Month"] = match.group(1)[:3]
        cur_book["Day"] = int(match.group(2))
        cur_book["Year"] = int(match.group(3))
        print(cur_book)
    else:
        # Regular book entry
        parts = line.split(" by ")
        current_book["Name"] = parts[0][len("".join(book_data)) + 3:].strip()
        current_book["Author"] = parts[1].strip()
        date_parts = parts[2].split(' ')
        current_book["Month"] = date_parts[3]
        current_book["Day"] = date_parts[4][:-1]
        current_book["Year"] = date_parts[5]

The Republic of Thieves: Gentleman Bastards" by Scott Lynch | audiobook on January 5, 2023
The Republic of Thieves: Gentleman Bastards" by Scott Lynch   on January 5, 2023
The Republic of Thieves: Gentleman Bastards Scott Lynch 
{'Name': 'The Republic of Thieves: Gentleman Bastards', 'Author': 'Scott Lynch', 'Audiobook': True}
{'Name': 'The Republic of Thieves: Gentleman Bastards', 'Author': 'Scott Lynch', 'Audiobook': True, 'Month': 'Jan', 'Day': 15, 'Year': 2023}
Hyperion" by Dan Simmons | audiobook on January 13, 2023
Hyperion" by Dan Simmons   on January 5, 2023
Hyperion Dan Simmons 
{'Name': 'Hyperion', 'Author': 'Dan Simmons', 'Audiobook': True, 'Month': 'Jan', 'Day': 15, 'Year': 2023}
{'Name': 'Hyperion', 'Author': 'Dan Simmons', 'Audiobook': True, 'Month': 'Jan', 'Day': 15, 'Year': 2023}
Propaganda" by Edward Bernays | audiobook on January 17, 2023
Propaganda" by Edward Bernays   on January 5, 2023
Propaganda Edward Bernays 
{'Name': 'Propaganda', 'Author': 'Edward Bernays', 'A

NameError: name 'current_book' is not defined

In [ ]:
# Create a pandas DataFrame for books
books_df = pd.DataFrame(book_data, columns=["Name", "Author", "Audiobook", "Month", "Day", "Year"])
books_df["Audiobook"] = books_df["Audiobook"].apply(lambda x: True if x else False)

# Print the DataFrame
print("Books DataFrame:")
books_df

Books DataFrame:


,Name,Author,Audiobook,Month,Day,Year


In [ ]:
# Find the 'Books:' section using the regular expression
books_section = re.search(r"Books:(.*?)(?=(?:Movies|Games):|$)", content, re.DOTALL).group(1).strip()
# Define a regular expression pattern for extracting book information
book_pattern = re.compile(r'\d+\. "([^"]+)" by ([^|]+)(?: \| ([^|]+))? on ([A-Za-z]+) (\d{1,2}), (\d{4})')
# Extract book information using the pattern
book_data = book_pattern.findall(books_section)
# Create a pandas DataFrame for books
books_df = pd.DataFrame(book_data, columns=["Name", "Author", "BKind", "Month", "Day", "Year"])
# Print the DataFrame
print("Books DataFrame:")
books_df

Books DataFrame:


,Name,Author,BKind,Month,Day,Year
0,The Republic of Thieves: Gentleman Bastards,Scott Lynch,audiobook,January,5,2023
1,Hyperion,Dan Simmons,audiobook,January,13,2023
2,Propaganda,Edward Bernays,audiobook,January,17,2023
3,The Fall of Hyperion,Dan Simmons,"audiobook on January 23, 2023\n5. ""Джури-харак...",January,23,2023
4,Zen in the Art of Writing,Ray Bradbury,audiobook,January,25,2023
5,"The Final Empire: Mistborn, Book 1",Brandon Sanderson,audiobook,February,2,2023
6,"The Well of Ascension: Mistborn, Book 2",Brandon Sanderson,audiobook,February,4,2023
7,The Hero of Ages: Mistborn: Book 3,Brandon Sanderson,audiobook,February,6,2023
8,Factotum,Charles Bukowski,audiobook,February,9,2023
9,Ham on Rye,Charles Bukowski,"audiobook on February 12, 2023\n12. ""Warbreake...",February,13,2023


In [ ]:
# Find the 'Books:' section using the regular expression
books_section = re.search(r"Books:(.*?)(?=\n\n|$)", content, re.DOTALL).group(1).strip()

# Define the regular expression pattern
book_pattern = re.compile(r'"([^"]+)" by ([^\|]+) \| audiobook|on ([A-Za-z]+) (\d{1,2}), (\d{4})')

# Extract book information using the pattern
book_data = book_pattern.findall(books_section)
print(book_data)
# Create a pandas DataFrame for books
books_df = pd.DataFrame(book_data, columns=["Name", "Author", "Audiobook", "Month", "Day", "Year"])

# Convert the 'Audiobook' column to boolean
books_df["Audiobook"] = books_df["Audiobook"].apply(lambda x: True if x else False)

# Print the DataFrame
print("Books DataFrame:")
print(books_df)


[('The Republic of Thieves: Gentleman Bastards', 'Scott Lynch', '', '', ''), ('', '', 'January', '5', '2023'), ('Hyperion', 'Dan Simmons', '', '', ''), ('', '', 'January', '13', '2023'), ('Propaganda', 'Edward Bernays', '', '', ''), ('', '', 'January', '17', '2023'), ('The Fall of Hyperion', 'Dan Simmons', '', '', ''), ('', '', 'January', '23', '2023'), ('Джури-характерники', 'Володимир Рутківський on January 23, 2023\n6. "Zen in the Art of Writing" by Ray Bradbury', '', '', ''), ('', '', 'January', '25', '2023'), ('The Final Empire: Mistborn, Book 1', 'Brandon Sanderson', '', '', ''), ('', '', 'February', '2', '2023'), ('The Well of Ascension: Mistborn, Book 2', 'Brandon Sanderson', '', '', ''), ('', '', 'February', '4', '2023'), ('The Hero of Ages: Mistborn: Book 3', 'Brandon Sanderson', '', '', ''), ('', '', 'February', '6', '2023'), ('Factotum', 'Charles Bukowski', '', '', ''), ('', '', 'February', '9', '2023'), ('Ham on Rye', 'Charles Bukowski', '', '', ''), ('', '', 'February', '

ValueError: 6 columns passed, passed data had 5 columns

In [ ]:
# Find the 'Books:' section using the regular expression
books_section = re.search(r"Books:(.*?)(?=Movies:|Games:)", content, re.DOTALL).group(1).strip()
# Define a regular expression pattern for extracting book information
book_pattern = re.compile(r'\d+\. "([^"]+)" by ([^|]+) \| (audiobook) on ([A-Za-z]+) (\d{1,2}), (\d{4})|'
                          r'\d+\. "([^"]+)" by ([^|]+) on ([A-Za-z]+) (\d{1,2}), (\d{4})')
# Extract book information using the pattern
book_data = book_pattern.findall(books_section)
print(book_data)
# Create a pandas DataFrame for books
book_columns = ["Name", "Author", "Audiobook", "Month", "Day", "Year"]
books_df = pd.DataFrame(book_data, columns=book_columns)
books_df["Audiobook"] = books_df["Audiobook"].apply(lambda x: x == "audiobook")
# Print the DataFrame
print("Books DataFrame:")
books_df

[('The Republic of Thieves: Gentleman Bastards', 'Scott Lynch', 'audiobook', 'January', '5', '2023', '', '', '', '', ''), ('Hyperion', 'Dan Simmons', 'audiobook', 'January', '13', '2023', '', '', '', '', ''), ('Propaganda', 'Edward Bernays', 'audiobook', 'January', '17', '2023', '', '', '', '', ''), ('The Fall of Hyperion', 'Dan Simmons', 'audiobook', 'January', '23', '2023', '', '', '', '', ''), ('Джури-характерники', 'Володимир Рутківський on January 23, 2023\n6. "Zen in the Art of Writing" by Ray Bradbury', 'audiobook', 'January', '25', '2023', '', '', '', '', ''), ('The Final Empire: Mistborn, Book 1', 'Brandon Sanderson', 'audiobook', 'February', '2', '2023', '', '', '', '', ''), ('The Well of Ascension: Mistborn, Book 2', 'Brandon Sanderson', 'audiobook', 'February', '4', '2023', '', '', '', '', ''), ('The Hero of Ages: Mistborn: Book 3', 'Brandon Sanderson', 'audiobook', 'February', '6', '2023', '', '', '', '', ''), ('Factotum', 'Charles Bukowski', 'audiobook', 'February', '9', 

ValueError: 6 columns passed, passed data had 11 columns